In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.decomposition import PCA

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_features_dtypes = {"cp_type": "category","cp_dose": "category"}

In [ ]:
train_features = pd.read_csv('/kaggle/input/lish-moa/train_features.csv',dtype = train_features_dtypes)

In [ ]:
train_features['train'] = 1

In [ ]:
test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv',dtype = train_features_dtypes)

In [ ]:
test_features['train'] = 0

In [ ]:
temp = pd.concat([train_features,test_features],axis = 0)

In [ ]:
del train_features,test_features

In [ ]:
temp.shape

In [ ]:
for col, col_dtype in train_features_dtypes.items():
    if col_dtype == "category":
        temp[col] = temp[col].cat.codes.astype("int16")
        temp[col] -= temp[col].min()

In [ ]:
train_features = temp[temp['train'] == 1].copy()
test_features = temp[temp['train'] == 0].copy()
del temp

In [ ]:
train_features = train_features.drop(columns='train')
test_features = test_features.drop(columns='train')

In [ ]:
test_features.shape

In [ ]:
train_features.shape

In [ ]:
train_features.head()

In [ ]:
train_features[['cp_type']].info()

In [ ]:
train_targets_scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')

In [ ]:
list(train_targets_scored.columns)[:5]

In [ ]:
train_targets_scored.shape

In [ ]:
train_targets_scored.head()

In [ ]:
df = pd.merge(train_features,train_targets_scored,how='inner',on='sig_id')

In [ ]:
df.shape

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

In [ ]:
sample_submission.shape

In [ ]:
sample_submission.head()

In [ ]:
submission = sample_submission.copy()
for col in submission.columns[1:]:
    submission[col].values[:] = 0


In [ ]:
X_cols = list(train_features.drop('sig_id',axis=1).columns)

In [ ]:
X_cols[:5]

In [ ]:
y_cols = list(train_targets_scored.drop('sig_id',axis=1).columns)

### Feature Engineering

In [ ]:
stack = pd.concat([train_features,test_features],axis=0)

In [ ]:
stack.shape

In [ ]:
# %%time
def fe_woe(y):
    
    new_X_cols = X_cols.copy()
    df1 = df.copy()
    test_features1 = test_features.copy()
    for i in X_cols[3:]:  #['c-98'] 
        print(i)
        quack, bins =  pd.qcut(stack[i], 10, retbins=True, duplicates='drop')
        df1[i + '_cut'] = pd.cut(df1[i], bins=bins, labels=False, include_lowest = True)
        woe = df1.groupby([i + '_cut'])[y].agg(['count','sum']).reset_index()
        woe['non_event'] = woe['count'] - woe['sum']
        woe['pct_event'] = woe['sum'] / woe['sum'].sum() 
        woe['pct_non_event'] = np.where(woe['non_event'].sum() == 0, 0, woe['non_event'] / woe['non_event'].sum())
        woe[i + '_woe'] =  np.where((woe['non_event'].sum() == 0), 0, np.log((woe['pct_event'] + 0.5) / (woe['pct_non_event'] + 0.5)))
        woe[i + '_iv'] =  (woe['pct_event'] - woe['pct_non_event']) * woe[i + '_woe']
        iv =woe[i + '_iv'].sum()
        #print(iv)
        if iv >= 0.15:
            try_1 = woe[[i + '_cut',i + '_woe']].to_dict()
            #df = df.merge(woe[[i + '_cut',i + '_woe']],how='left',on=[i + '_cut'])
            df1[i + '_woe'] = df1[i + '_cut'].map(try_1[i + '_woe'])
            df1 = df1.drop(i + '_cut',axis=1)
            test_features1[i + '_cut'] = pd.cut(test_features1[i], bins=bins, labels=False, include_lowest = True)  #
            #test_features = test_features.merge(woe[[i + '_cut',i + '_woe']],how='left',on=[i + '_cut'])
            test_features1[i + '_woe'] = test_features1[i + '_cut'].map(try_1[i + '_woe'])
            test_features1 = test_features1.drop(i + '_cut',axis=1)
            new_X_cols = new_X_cols + [i + '_woe']
        else:
            1==1
            
    #scaler = StandardScaler()
    #df1 = scaler.fit_transform(df1[new_X_cols])
    df1 = scale(df1[new_X_cols],new_X_cols)
    # X_test_scaler = scaler.transform(X_test)
    #test_features1 = scaler.transform(test_features1[new_X_cols]) 
    test_features1 = scale(test_features1[new_X_cols],new_X_cols)
    
    return df1, test_features1

In [ ]:
def scale(a1,cols):
    scaler = StandardScaler()

    a2 = scaler.fit_transform(a1)
    a3 = pd.DataFrame(a2)
    a3.columns = cols

    return a3

### Model

The core data structures of Keras are layers and models.
The Keras functional API is a way to create models that are more flexible than the tf.keras.Sequential API. The functional API can handle models with non-linear topology, shared layers, and even multiple inputs or outputs.

The main idea is that a deep learning model is usually a directed acyclic graph (DAG) of layers. So the functional API is a way to build graphs of layers.




In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
import tensorflow.keras.backend as K

from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.losses import BinaryCrossentropy
import tensorflow_addons as tfa
def create_model(num_columns, actv='relu'):
    model = tf.keras.Sequential([tf.keras.layers.Input(num_columns)])
                
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tfa.layers.WeightNormalization(tf.keras.layers.Dense(1024, activation=actv)))
    
    if actv == 'elu':
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.AlphaDropout(0.2))
        model.add(tfa.layers.WeightNormalization(tf.keras.layers.Dense(512, kernel_initializer='lecun_normal', activation='selu')))
    else:
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dropout(0.2))
        model.add(tfa.layers.WeightNormalization(tf.keras.layers.Dense(1024, activation=actv))) 

    #============ Final Layer =================
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tfa.layers.WeightNormalization(tf.keras.layers.Dense(206, activation="sigmoid")))
    
    model.compile(optimizer=tfa.optimizers.AdamW(lr = 1e-3, weight_decay = 1e-5, clipvalue = 756), 
                  loss=BinaryCrossentropy(label_smoothing=1e-15),
                  )
    return model

In [ ]:
nfolds=5

# prepare split
kf = KFold(n_splits = nfolds)

# base model definition throught sklearn Pipeline
pca = PCA(n_components = 300)
# svm0 = SVC(C = 0.1,probability =True)


# base_model = Pipeline(steps=[('pca', pca), ('nn', model)])

#mo_base = MultiOutputClassifier(model, n_jobs=-1) #base_model
xtrain = df[X_cols]#.head(1000)  set to small value for testing code
ytrain = df[y_cols]#.head(1000) set to small value for testing code

xtest = test_features[X_cols]

#standard scaler
xtrain = scale(xtrain,X_cols)
xtest = scale(xtest,X_cols)

# storage matrices for OOF / test predictions
prval = np.zeros(ytrain.shape)
#kfold cv 
z = pd.DataFrame()
for (ff, (id0, id1)) in enumerate(kf.split(xtrain)):
     
    x0, x1 = xtrain.loc[id0], xtrain.loc[id1]
    y0, y1 = np.array(ytrain.loc[id0]), np.array(ytrain.loc[id1])
    
    # fix for empty columns
    check_for_empty_cols = np.where(y0.sum(axis = 0) == 0)[0]
    if len(check_for_empty_cols):
        y0[0,check_for_empty_cols] = 1
    
    model1 = create_model(len([i for i in range(x0.shape[1])]))
    # fit model
    model1.fit(x0,y0) #fails here
    
    # predicitons
    prv = model1.predict_proba(x1)  #[:, 1] see note below, this does not appear to work on a multioutput scenario
    prf = model1.predict_proba(xtest)  #[:, 1]
    
    z = pd.concat([z,pd.DataFrame(prv)],axis=0)
    
    submission[y_cols] += prf / nfolds

In [ ]:
# keras.utils.plot_model(model1, "my_first_model.png")


In [ ]:
# xtrain

In [ ]:
# len([i for i in range(x0.shape[1])])

In [ ]:
prf.shape

In [ ]:
# x0.shape[-1]

In [ ]:
def log_loss_metric_ind(y_true, y_pred):
    y_pred_clip = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = - np.mean(np.mean(y_true * np.log(y_pred_clip) + (1 - y_true) * np.log(1 - y_pred_clip)))
    return loss

In [ ]:
prval_df = pd.DataFrame(z)

In [ ]:
prval_df.columns = y_cols

In [ ]:
#highlight the worst performing models
perf_check = []
for i in y_cols:
    perf_check.append((i,log_loss_metric_ind(ytrain[i], prval_df[i])))

In [ ]:
results = pd.DataFrame(perf_check)

In [ ]:


results.columns = ['target','log_loss']



In [ ]:
#worst performing models
results.sort_values('log_loss',ascending=False).head(20)

In [ ]:
results['log_loss'].describe()

In [ ]:
results.sort_values('log_loss',ascending=False).head(10)['target']

In [ ]:
# %%time
# #from sklearn.model_selection import StratifiedKFold


# #clf = make_pipeline(StandardScaler(), SVC(kernel='rbf', probability =True))
# #clf = SVC(kernel='rbf',C= 0.001,  probability =True) # Platt calibration
# z1 = pd.DataFrame()
# for y in results.sort_values('log_loss',ascending=False).head(10)['target'] : #y_cols ['5-alpha_reductase_inhibitor','11-beta-hsd1_inhibitor']
#     df1, test_features1 = fe_woe(y)
#     #df1, test_features1 = scale()
#     print(y)
#     print("step1 passed")

#     for (ff, (id0, id1)) in enumerate(kf.split(df1)):

#         x0, x1 = df1.loc[id0], df1.loc[id1]
#         y0, y1 = np.array(ytrain.loc[id0]), np.array(ytrain.loc[id1])

#         # fix for empty columns
#         check_for_empty_cols = np.where(y0.sum(axis = 0) == 0)[0]
#         if len(check_for_empty_cols):
#             y0[0,check_for_empty_cols] = 1


#         print("step2 passed")

#         #clf.fit(X_train, y_train)
#         #pred = clf.predict_proba(test_features1)
#         #submission[y] += pred[:,1] / splits
#         model2 = create_model(len([i for i in range(x0.shape[1])]))
#         print("step3 passed")
#         model2.fit(x0,y0) 
#         print("step4 passed")
#         pred = model2.predict_proba(test_features1)
#         prv1 = model2.predict_proba(x1)  #[:, 1]
#         z1 = pd.concat([z1,pd.DataFrame(prv1)],axis=0)
#         print("step5 passed")
#         submission[y] = 0
#         print("step6 passed")
#         submission[y] += np.array(pd.DataFrame(pred)[ytrain.columns.get_loc(y)]) / nfolds #[:,1]
#         print("step7 passed")

In [ ]:
submission.shape

In [ ]:
submission.head()

In [ ]:
submission.shape

In [ ]:
submission[i].mean()

In [ ]:
submission.head()

In [ ]:

#test_export = test.loc[:, ['id', 'target']]
submission.to_csv('submission.csv', index=False)